## This notebook shows all the means to select some spots

In [ ]:
%matplotlib inline

import time

import torch

from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram  # you don't need to import more to access all the filters

### Firstly, we have to iniitialize a diagram with a lot of spots
* For more details about the ways to create a Diagram, please have a look on the `api_init_diagram` notebook.

In [ ]:
file = next(get_samples().iterdir())
diag_ref = Diagram(file)
diag_ref.find_spots()  # see `peaks_search` for details

### Secondly, let filtering!

In [ ]:
"""Very basic slow examples for demonstration."""

# first) Having an idear of the initial diagram
diag1 = diag_ref.clone()
print("*** before filtering ***")
print(diag1)

# 1) Filtering by indexs
indexs_to_keep = list(range(0, len(diag1.spots), 2))
diag2 = diag1.filter_spots(indexs_to_keep, "keep even spots from indexs")

# 2) filtering by mask
mask_to_keep = [True if i%2 == 0 else False for i in range(len(diag2.spots))]
diag3 = diag2.filter_spots(mask_to_keep, "keep even spots from mask")

# 3) sorted the spots
criteria = diag3.compute_pxl_intensities()
sorted_indexs = torch.argsort(criteria, descending=True)
diag4 = diag3.filter_spots(sorted_indexs, "sorted spots by intensities")

# final) Resume current state
print("*** after filtering ***")
print(diag4)

In [ ]:
"""Same examples with no fork and inplace (more efficient)."""

# first) Having an idear of the initial diagram
diag = diag_ref.clone()
print("*** before filtering ***")
print(diag)

# 1) Filtering by indexs
indexs_to_keep = torch.arange(0, len(diag.spots), 2)
diag.filter_spots(indexs_to_keep, "keep even spots from indexs", inplace=True)

# 2) filtering by mask
mask_to_keep = torch.arange(len(diag.spots))%2 == 0
diag.filter_spots(mask_to_keep, "keep even spots from mask", inplace=True)

# 3) sorted the spots
criteria = diag.compute_pxl_intensities()
sorted_indexs = torch.argsort(criteria, descending=True)
diag.filter_spots(sorted_indexs, "sorted spots by intensities", inplace=True)

# final) Resume current state
print("*** after filtering ***")
print(diag)

### Thridly, compare speed

In [ ]:
"""Timing the means to filter."""

# exec the cell twice

INDEXS = torch.arange(0, len(diag_ref.spots), 2)
MASK = torch.arange(len(diag_ref.spots)) % 2 == 0


def filter_inplace_index(diag):
    diag.filter_spots(INDEXS, "keep even spots from indexs", inplace=True)

def filter_inplace_mask(diag):
    diag.filter_spots(MASK, "keep even spots from mask", inplace=True)

def filter_copy_index(diag):
    diag = diag.filter_spots(INDEXS, "keep even spots from indexs")

def filter_copy_mask(diag):
    diag = diag.filter_spots(MASK, "keep even spots from mask")

def timer(func):
    duration = torch.inf
    for _ in range(5):
        diags = [diag_ref.clone() for _ in range(50)]
        t_start = time.time()
        for diag_ in diags:
            func(diag_)
        duration = min(duration, time.time()-t_start)
    return duration

time_copy_index = timer(filter_copy_index)
time_inplace_index = timer(filter_inplace_index)
time_copy_mask = timer(filter_copy_mask)
time_inplace_mask = timer(filter_inplace_mask)
print(f"inplace (index) is {time_copy_index/time_inplace_index:.1f} times faster than copy")
print(f"inplace (mask) is {time_copy_mask/time_inplace_mask:.1f} times faster than copy")
print(f"index (inplace) is {time_inplace_index/time_inplace_mask:.2f} times faster than mask")
print(f"index (copy) is {time_copy_index/time_copy_mask:.2f} times faster than mask")